In [1]:
import os
import pandas as pd
import numpy as np

# Loading Data_Info

In [10]:
files = os.listdir('../../Further_Patches/')

In [11]:
files = pd.DataFrame(data=files, columns=['filename'])

In [12]:
#FOR TEST TCGA
x = files["filename"].str.split("_", n = 4, expand = True)
files["SVS_NO"] = x[x.columns[-4]]
files["ROW"] = x[x.columns[-3]]
files["COL"] = x[x.columns[-2]]
files["ROW"] = files["ROW"].astype('int')
files["COL"] = files["COL"].astype('int')
y = x[x.columns[-1]].str.split(".", n = 1, expand = True)
files["Class_Name"] = y[y.columns[0]]
files["Class_Name"] = files["Class_Name"].astype('int')
x = files["SVS_NO"].str.split("-", n = 5, expand = True)
files["SVS_NO"] = x[x.columns[2]]

In [4]:
#FOR Train bach
x = files["filename"].str.split("_", n = 3, expand = True)
y = x[x.columns[3]].str.split(".", n = 1, expand = True)
files["SVS_NO"] = x[x.columns[0]]
files["Class_Name"] = y[y.columns[0]]
files["Class_Name"] = files["Class_Name"].astype('int')

In [13]:
files['filename'] = '../../Further_Patches/' + files['filename']

In [14]:
files.head()

,filename,SVS_NO,ROW,COL,Class_Name
0,../../Further_Patches/TCGA-BH-A0BD-01Z-00-DX1....,A0BD,3,92,0
1,../../Further_Patches/TCGA-BH-A0BD-01Z-00-DX1....,A0BD,146,42,0
2,../../Further_Patches/TCGA-BH-A0BD-01Z-00-DX1....,A0BD,148,72,-1
3,../../Further_Patches/TCGA-BH-A0BO-01Z-00-DX1....,A0BO,65,124,0
4,../../Further_Patches/TCGA-BH-A0BO-01Z-00-DX1....,A0BO,63,95,0


In [15]:
print ("Total_Patches : ", files.shape[0])

Total_Patches :  73815


In [16]:
all_avalable_svs = np.unique(files.SVS_NO.astype('str'), return_counts=True)

In [17]:
all_avalable_svs

(array(['A0BD', 'A0BO', 'A0CQ'], dtype=object), array([38247, 30033,  5535]))

# Setting up Dryad Label for Invasive and Normal

In [19]:
#files['Class_Name'][files['Class_Name'] > 0] = 1 #FOR TRAIN BACH
files['Class_Name'][files['Class_Name'] < 0] = 1 #FOR TEST TCGA

/home/qali/anaconda3/envs/py37_gpu_env/lib/python3.5/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [20]:
np.unique(files.SVS_NO.astype('str'))

array(['A0BD', 'A0BO', 'A0CQ'], dtype=object)

In [21]:
training_all_avalable_svs = np.unique(files.SVS_NO.astype('str'), return_counts=True)
training_all_avalable_svs

(array(['A0BD', 'A0BO', 'A0CQ'], dtype=object), array([38247, 30033,  5535]))

In [22]:
print ("Training Total_Patches : ", files.shape[0])

Training Total_Patches :  73815


# S-Score Calculation (Metric used in ICIAR 2018)

In [23]:
def vectorized_s_score(x, y):
    a = 0.0000000001
    nom = np.sum(np.abs(x-y))
    xbin = x.copy()
    ybin = y.copy()
    xbin[xbin!=0] = 1.0
    ybin[ybin!=0] = 1.0
    temp = 1.0 - ((1.0 - xbin) * (1.0 - ybin))
    v = np.abs(y - 3)
    max_term = np.max(np.column_stack([y, v]), axis=1)
    temp1 = max_term * temp
    denom = np.sum(temp1)
    final = nom / (denom + a)
    return (1 - final)

# Loading Libraries for Training

In [24]:
import os
import json
import cv2
import numpy as np
import pandas as pd
import keras
import pandas as pd
import numpy as np
import os
from glob import glob
import itertools
import fnmatch
import random
import matplotlib.pylab as plt
import cv2
from scipy.misc import imresize, imread
import sklearn
from sklearn import model_selection
from sklearn.model_selection import train_test_split, KFold, cross_val_score, StratifiedKFold, learning_curve, GridSearchCV
from sklearn.metrics import confusion_matrix, make_scorer, accuracy_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
import keras
from keras import layers
from tqdm import tqdm
from keras import backend as K
from keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from keras.models import Sequential, model_from_json,load_model,Model
from keras.optimizers import SGD, RMSprop, Adam, Adagrad, Adadelta
from keras.utils import layer_utils
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, UpSampling2D , Conv2D, MaxPool2D, MaxPooling2D,Input,Concatenate,GlobalAveragePooling2D,GlobalMaxPooling2D,ZeroPadding2D,AveragePooling2D,Reshape,merge,Convolution2D
%matplotlib inline
from keras import layers
from keras.applications import MobileNetV2
from keras.callbacks import Callback, ModelCheckpoint, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.optimizers import Adam
import tensorflow as tf
from tqdm import tqdm
import efficientnet.keras as efn 
from keras.layers import Dense
from keras.optimizers import Adam, Nadam
from keras_radam import RAdam
from keras import backend as K
from sklearn.utils import class_weight

Using TensorFlow backend.
/home/qali/anaconda3/envs/py37_gpu_env/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/qali/anaconda3/envs/py37_gpu_env/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/qali/anaconda3/envs/py37_gpu_env/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/qa

# Training

In [25]:
#To set class_mode of generator we need to change label from dtype integer to dtype string
files['Class_Name'] = files['Class_Name'].astype('str')

In [26]:
def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


In [27]:
model = load_model('Complete_model_effnet_b0_5.h5', compile=False)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnet-b0 (Model)      (None, 1280)              4049564   
_________________________________________________________________
dense_1 (Dense)              (None, 128)               163968    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 65        
Total params: 4,221,853
Trainable params: 4,179,837
Non-trainable params: 42,016
_________________________________________________________________


# TEST

In [28]:
BATCH_SIZE = 1
def create_test_gen(Xfiles):
    return ImageDataGenerator(rescale=1/255.).flow_from_dataframe(
        Xfiles,
        directory='.',
        x_col='filename',
        class_mode=None,
        target_size=(256, 256),
        batch_size=BATCH_SIZE,
        shuffle=False
    )



In [29]:
for each in training_all_avalable_svs[0]:
    Xfiles = files[files.SVS_NO == each].copy()
    test_gen = create_test_gen(Xfiles)
    predictions = model.predict_generator(test_gen, steps=len(test_gen), verbose=1)
    #pred = np.argmax(predictions, axis=1)
    pred = predictions.copy()
    pred[pred > 0.5] = 1
    pred[pred <= 0.5] = 0
    print (accuracy_score(Xfiles.Class_Name.astype('int'), pred))
    print (f1_score(Xfiles.Class_Name.astype('int'), pred.astype('int'), average='micro'))
    print (f1_score(Xfiles.Class_Name.astype('int'), pred.astype('int'), average='macro'))
    print (f1_score(Xfiles.Class_Name.astype('int'), pred.astype('int'), average='weighted'))
    print (confusion_matrix(Xfiles.Class_Name.astype('int'), pred.astype('int')))
    Xfiles['predictions_epochs_3'] = predictions.copy()
    Xfiles.to_csv(each + '_predictions.csv', index=False)

Found 38247 validated image filenames.
38247/38247 [==============================] - 335s 9ms/step
0.9176928909456951
0.9176928909456951
0.7107440995537905
0.9020330769885566
[[33725   350]
 [ 2798  1374]]
Found 30033 validated image filenames.
30033/30033 [==============================] - 258s 9ms/step
0.9850497785768988
0.9850497785768988
0.8832636528646592
0.9853800644647827
[[28814   270]
 [  179   770]]
Found 5535 validated image filenames.
5535/5535 [==============================] - 48s 9ms/step
0.9192411924119241
0.9192411924119241
0.714265041868574
0.9212962007471496
[[4888  247]
 [ 200  200]]
